In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
# A convnet takes as input tensors of shape (image_height, image_width, image_channels)-- not including the batch dimensions. 
# The dimensions below are formatted for MNIST images

In [3]:
inputs = keras.Input(shape=(28,28,1))

In [4]:
# The number of channels is controlled via filters--(32, 64, 128)

In [5]:
#Convolutional layers learn local patterns--in the case of images, patterns found in small 2D windows
#of the inputs
# CONVNETS have two interesting properties
# 1. The patterns that they learn are translation-invariant 
#    After learning a certain pattern, say in the lower right corner of a pic, a convnet can 
#    recognize it anywhere, such as the top left corner. A densely connected model would have
#    have to learn the pattern anew if it appeared at a new location. This makes convnets
#    data efficient when processing images: they need fewer training samples to learn
#    representations that have generalization power
# 2. They can learn spatial hiearchies of patterns 
#    A first CNN layer will learn small local patterns such as edges, a second larger patterns
#    made of features of the first layers, and so on. Allows CNNs to learn increasingly complex
#    and abstract concepts, because the visual worls is fundamentally spatially hiearchial

x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)

# DENSE layers learn global patterns in their input feature space -- for MNIST this means all pixels
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Metal device set to: Apple M1 Max


2021-12-10 21:25:53.901841: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-10 21:25:53.902335: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0     

In [7]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [8]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255

In [9]:
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

In [10]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [11]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

2021-12-10 21:26:08.211484: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-10 21:26:08.211651: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-10 21:26:08.342184: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/5
938/938 [==============================] - 9s 10ms/step - loss: 0.1558 - accuracy: 0.9514
Epoch 2/5
938/938 [==============================] - 9s 10ms/step - loss: 0.0435 - accuracy: 0.9864
Epoch 3/5
938/938 [==============================] - 9s 10ms/step - loss: 0.0296 - accuracy: 0.9903
Epoch 4/5
938/938 [==============================] - 9s 10ms/step - loss: 0.0229 - accuracy: 0.9931
Epoch 5/5
938/938 [==============================] - 9s 10ms/step - loss: 0.0178 - accuracy: 0.9950


In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

 24/313 [=>............................] - ETA: 1s - loss: 0.0257 - accuracy: 0.9870   

2021-12-10 21:27:03.822599: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 5ms/step - loss: 0.0242 - accuracy: 0.9914


In [13]:
print(f'Test accuracy: {test_acc: .3f}')

Test accuracy:  0.991
